# Set up 

In [2]:
#!pip install beautifulsoup4
#!pip install requests
#!pip install urllib2
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime

# Extract all the links under home category 

In [ ]:
from bs4 import BeautifulSoup, SoupStrainer
import requests

url = "http://www.zoolert.com/"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)
linkal=[]
for link in soup.find_all('a'):
    linkal.append(link.get('href'))
linkal

In [194]:
link_home=[]
for link in linkal:
    if re.findall('/home/', link):
        link_home.append(link)
link_home= set(link_home)
link_home= pd.DataFrame(list(link_home), columns= list(['url']))
link_home['prod_name'] = link_home['url'].str.split('/').str[-2]
link_home.to_csv('product_name_home.csv')

In [196]:
link_home[1:5]

,url,prod_name
1,/home/oximeter/,oximeter
2,/home/emergen-c-vitamin/,emergen-c-vitamin
3,/home/all-purpose-cleaner/,all-purpose-cleaner
4,/home/floor-sweeper-pads/,floor-sweeper-pads


# Read in data from the home file

In [3]:
link_home = pd.read_csv('product_name_home.csv')

In [8]:
def get_histchange( prod_name):
    URL = 'https://www.zoolert.com/home/'+ prod_name+ '/instockhistory.php'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find(id=='div')
    lists = results.find_all('tbody')
    for tr in lists:
        a = [td for td in tr.stripped_strings]
    nrow = int(len(a)/4)
    df = pd.DataFrame(np.array(a).reshape(nrow,4), columns = list(['retailer', 'name', 'time_update', 'duration']))
    df['price']= df["name"].str.split("--", n = 1, expand = True)[1]
    df['product'] = prod_name
    print(prod_name)
    return df;

In [10]:
history_df= pd.DataFrame()
temp =  pd.DataFrame()
for prod_name in link_home["prod_name"]:
    temp= get_histchange(prod_name)
    history_df = pd.concat([history_df, temp])

disposable-laex-gloves
oximeter
emergen-c-vitamin
all-purpose-cleaner
floor-sweeper-pads
laundry
weight-training
hand-sanitizers
respirator-face-mask
hibiclens-antiseptic-skin-cleanser
bread-maker
sewing-machine
rice
thermoscan-ear-lens-filters
noodles-cups
trampolines
antibacterial-hand-soap
hair-cutting-kit
toilet-bidet-attachment-water-sprayer
isopropyl-alcohol
disinfecting-wipes
hydrogen-peroxide
inflatable-pool
baby-wipes
flour
shelf-stable-milk
emergency-food-supply
paper-towels
facial-tissues
toilet-paper
over-the-counter-medicines
thermometer


In [30]:
history_df[1:10]
history_df.groupby(['product']).size()

product
all-purpose-cleaner                      600
antibacterial-hand-soap                  600
baby-wipes                               600
bread-maker                               59
disinfecting-wipes                       600
disposable-laex-gloves                   473
emergen-c-vitamin                        600
emergency-food-supply                    147
facial-tissues                           601
floor-sweeper-pads                       103
flour                                    277
hair-cutting-kit                         417
hand-sanitizers                          600
hibiclens-antiseptic-skin-cleanser        21
hydrogen-peroxide                         45
inflatable-pool                           30
isopropyl-alcohol                        601
laundry                                  203
noodles-cups                             600
over-the-counter-medicines               370
oximeter                                 132
paper-towels                             601
re

In [26]:

date= datetime.date(datetime.now())
history_df.to_csv('home_hist_'+str(date)+'.csv')